In [ ]:
# Reload the newly uploaded Excel file after session reset
new_file_path = '/mnt/data/R3 L1 NM cellular distribution_8 bit.xlsx'
new_excel_data = pd.ExcelFile(new_file_path)
new_data_frames = {sheet_name: new_excel_data.parse(sheet_name) for sheet_name in new_excel_data.sheet_names}

# Re-define the condition logic function
def pivot_and_process_conditions(data):
    pivoted_data = data.pivot_table(
        index=["Region1", "Region", "ROI"],
        columns="Channel",
        values="Max",
        aggfunc="max"
    ).reset_index()
    
    pivoted_data["D1DR=255; D2DR=255; OXTR=255"] = ((pivoted_data.get("D1DR", 0) == 255) & (pivoted_data.get("D2DR", 0) == 255) & (pivoted_data.get("OXTR", 0) == 255)).astype(int)
    pivoted_data["D1DR=255; D2DR=0; OXTR=255"] = ((pivoted_data.get("D1DR", 0) == 255) & (pivoted_data.get("D2DR", 0) == 0) & (pivoted_data.get("OXTR", 0) == 255)).astype(int)
    pivoted_data["D1DR=0; D2DR=255; OXTR=255"] = ((pivoted_data.get("D1DR", 0) == 0) & (pivoted_data.get("D2DR", 0) == 255) & (pivoted_data.get("OXTR", 0) == 255)).astype(int)
    pivoted_data["D1DR=255; D2DR=255; OXTR=0"] = ((pivoted_data.get("D1DR", 0) == 255) & (pivoted_data.get("D2DR", 0) == 255) & (pivoted_data.get("OXTR", 0) == 0)).astype(int)
    pivoted_data["D1DR=255; D2DR=0; OXTR=0"] = ((pivoted_data.get("D1DR", 0) == 255) & (pivoted_data.get("D2DR", 0) == 0) & (pivoted_data.get("OXTR", 0) == 0)).astype(int)
    pivoted_data["D1DR=0; D2DR=255; OXTR=0"] = ((pivoted_data.get("D1DR", 0) == 0) & (pivoted_data.get("D2DR", 0) == 255) & (pivoted_data.get("OXTR", 0) == 0)).astype(int)
    pivoted_data["D1DR=0; D2DR=0; OXTR=255"] = ((pivoted_data.get("D1DR", 0) == 0) & (pivoted_data.get("D2DR", 0) == 0) & (pivoted_data.get("OXTR", 0) == 255)).astype(int)
    pivoted_data["D1DR=0; D2DR=0; OXTR=0"] = ((pivoted_data.get("D1DR", 0) == 0) & (pivoted_data.get("D2DR", 0) == 0) & (pivoted_data.get("OXTR", 0) == 0)).astype(int)
    
    return pivoted_data

# Process and combine all sheets
processed_data = {animal: pivot_and_process_conditions(data) for animal, data in new_data_frames.items()}
combined_data = pd.concat(processed_data, names=["Animal", "Index"]).reset_index(level="Index", drop=True)

# Summarize
summary = combined_data.groupby(["Region1", "Animal"]).agg(
    Total_ROIs=("ROI", "count"),
    Condition_1_Positive=("D1DR=255; D2DR=255; OXTR=255", "sum"),
    Condition_2_Positive=("D1DR=255; D2DR=0; OXTR=255", "sum"),
    Condition_3_Positive=("D1DR=0; D2DR=255; OXTR=255", "sum"),
    Condition_4_Positive=("D1DR=255; D2DR=255; OXTR=0", "sum"),
    Condition_5_Positive=("D1DR=255; D2DR=0; OXTR=0", "sum"),
    Condition_6_Positive=("D1DR=0; D2DR=255; OXTR=0", "sum"),
    Condition_7_Positive=("D1DR=0; D2DR=0; OXTR=255", "sum"),
    Condition_8_Positive=("D1DR=0; D2DR=0; OXTR=0", "sum"),
).reset_index()

# Rename and format as percentages with counts
condition_descriptions = {
    "Condition_1_Positive": "D1DR=255; D2DR=255; OXTR=255 (Count)",
    "Condition_2_Positive": "D1DR=255; D2DR=0; OXTR=255 (Count)",
    "Condition_3_Positive": "D1DR=0; D2DR=255; OXTR=255 (Count)",
    "Condition_4_Positive": "D1DR=255; D2DR=255; OXTR=0 (Count)",
    "Condition_5_Positive": "D1DR=255; D2DR=0; OXTR=0 (Count)",
    "Condition_6_Positive": "D1DR=0; D2DR=255; OXTR=0 (Count)",
    "Condition_7_Positive": "D1DR=0; D2DR=0; OXTR=255 (Count)",
    "Condition_8_Positive": "D1DR=0; D2DR=0; OXTR=0 (Count)",
}
summary.rename(columns=condition_descriptions, inplace=True)

# Create final percentage columns with formatted text
for condition_label in list(condition_descriptions.values()):
    percent_col = condition_label.replace("(Count)", "(%)").strip()
    summary[percent_col] = summary.apply(
        lambda row: f"{(row[condition_label] / row['Total_ROIs']) * 100:.2f}% ({row[condition_label]}/{row['Total_ROIs']})",
        axis=1
    )

# Drop count columns
summary.drop(columns=list(condition_descriptions.values()), inplace=True)

# Display final table
import ace_tools as tools; tools.display_dataframe_to_user(
    name="Final Summary Table from Uploaded File",
    dataframe=summary
)
